In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import json
import pandas as pd
from pandas import DataFrame as df 
from requests.exceptions import MissingSchema

In [2]:
class crawly:
    def __init__(self, key, url):
        self.key = key
        self.url = url
        self.sub_pg = []
    
    def crawl(self):
        if self.key == '{:04d}'.format(0): #Dada Dallas
            self.sub_pg=[]
            site = requests.get(self.url)
            site.encoding = 'ISO-885901'
            soup = BeautifulSoup(site.text, 'html.parser')
            page_list = soup.find_all(class_='event-name headliners')
            for i in page_list:
                cont = i.contents[0]
                link = cont['href']
                self.sub_pg.append(link)
        elif self.key =='{:04d}'.format(1):
            self.sub_pg=[]
            site = requests.get(self.url)
            site.encoding = 'ISO-885901'
            soup = BeautifulSoup(site.text, 'html.parser')
            page_list = soup.find_all("div", class_="bottom_btns")
            for i in page_list:
                cont = i.contents[1]
                self.sub_pg.append(i.contents[1]['href'])
        elif self.key =='{:04d}'.format(2):
            self.sub_pg =[]
            site = requests.get(self.url)
            site.encoding = 'ISO-885901'
            soup = BeautifulSoup(site.text, 'lxml')
            self.sub_pg = soup.find_all("a", class_="staff__item staff__item--five-columns")
        elif self.key =='{:04d}'.format(3):
            self.sub_pg=[]
            site = requests.get(self.url)
            site.encoding = 'ISO-885901'
            soup = BeautifulSoup(site.text, 'html.parser')
            page_list = soup.find("iframe", class_="iframe-class")
            site = requests.get(page_list.get('src'))
            site.encoding = 'ISO-885901'
            soup = BeautifulSoup(site.text, 'lxml')
            sub_par = soup.find('div', class_='aswidget')
            for i in sub_par.find_all("a"):
                self.sub_pg.append("http:"+i.get("href"))
        elif self.key =='{:04d}'.format(4):
            self.sub_pg=[]
            site = requests.get(self.url)
            site.encoding = 'ISO-885901'
            soup = BeautifulSoup(site.text, 'html.parser')
            page_list = soup.find_all("div", class_="grid-item medium-6 double columns")
            for i in soup.find_all("div", class_="grid-item medium-6 large-4 single columns"):
                page_list.append(i)
            page_list.append(soup.find("div", class_="grid-item medium-6 large-4 single columns end"))    
            for i in range(len(page_list)):
                self.sub_pg.append(page_list[i].contents[1].get("href"))
        
        else:
            print("Invalid Key")
            return[]
        return self.sub_pg

In [3]:
class creepy:
    prd = {  
        'Org Key':"",
        'Event Key':"",
        'Event Title' :"",
        'Topline': "",
        'Headliner':"",
        'Openers':"",
        'Date(s)':"",
        'Time(s)':"",
        'Price/Admission':'',
        'Age Restriction':'',
        'Event Description':'',
        'Staff/Artists':'',
        'Category':'',
        'Venue':'',
        'Venue Info':'',
        'Street Name':'',
        'Address Line 2':',',
        'City':'',
        'State':'',
        'Postal Code':'',
        'Event Image URL':"",
        'Venue Info':'',
        'Location Link':'',
        'Get Tickets':'',
        'URL':''}
    
    def __init__(self, key, pages):
        self.key = key
        self.pages = pages
        self.info = []
        self.count=0
        
    def creep(self, bonus=str):
        if self.key == '{:04d}'.format(0): #Dada Dallas
            for link in self.pages:
                event_info = creepy.prd.copy()
                on_sale = True
                event_info.update([('Org Key',self.key)])
                event_info.update([('Event Key','{:06d}'.format(self.count))])
                self.count+=1
                s_page = requests.get(link)
                #print(link) error finder
                sub_soup = BeautifulSoup(s_page.text, 'html.parser')
                sub_parse = sub_soup.find('meta', property ="og:title")
                event_info.update([('Event Title', sub_parse.attrs['content'])])
                sub_parse = sub_soup.find('section', class_="topline-info presented-by")
                if type(sub_parse) is type(None):
                    event_info.update([('Topline', "NA")])
                else:    
                    event_info.update([('Topline', sub_parse.get_text())])
                sub_parse = sub_soup.find('h1', class_="event-name headliners")
                event_info.update([('Headliner', sub_parse.get_text())])
                sub_parse = sub_soup.find('h2', class_="supports")
                if type(sub_parse) is type(None):
                    event_info.update([('Openers', 'NA')])
                else:
                    event_info.update([('Openers', sub_parse.get_text())])
                sub_parse = sub_soup.find('span', class_="dates")
                event_info.update([('Date(s)', sub_parse.get_text())])
                sub_parse = sub_soup.find('span', class_="start")
                event_info.update([('Time(s)', sub_parse.get_text())])
                sub_parse = sub_soup.find('span', class_="sales-ended inactive")
                if type(sub_parse) is type(None):
                    sub_parse = sub_soup.find('span', class_="price-range")
                    event_info.update([('Price/Admission', sub_parse.get_text())])
                else:
                    event_info.update([('Price/Admission', sub_parse.get_text())])
                    on_sale = False
                sub_parse = sub_soup.find('section', class_="age-restriction all-ages")
                if type(sub_parse) is type(None):
                    sub_parse = sub_soup.find('section', class_="age-restriction over-21")
                    event_info.update([('Age Restriction', sub_parse.get_text())])
                else:    
                    event_info.update([('Age Restriction', sub_parse.get_text())])
                sub_parse = sub_soup.find('article', class_="event-description")
                event_info.update([('Event Description', sub_parse.contents[1].text + " " +sub_parse.contents[3].text)])
                event_info.update([('Category', "Live Music, Night Life, Performance")])
                sub_parse = sub_soup.find('address', class_="venue-info")
                event_info.update([('Venue', sub_parse.contents[2])])
                event_info.update([('Venue Info', "Night Club")])
                sub_parse = sub_soup.find('meta', property="og:street-address")
                event_info.update([('Street Name', sub_parse.attrs['content'])])
                sub_parse = sub_soup.find('meta', property="og:locality")
                event_info.update([('City', sub_parse.attrs['content'])])
                sub_parse = sub_soup.find('meta', property="og:region")
                event_info.update([('State', sub_parse.attrs['content'])])
                sub_parse = sub_soup.find('meta', property="og:postal-code")
                event_info.update([('Postal Code', sub_parse.attrs['content'])])
                sub_parse = sub_soup.find('img')
                event_info.update([('Event Image URL', sub_parse.attrs['src'])])
                event_info.update([('URL',link)])
        
                if on_sale == True:
                    sub_parse = sub_soup.find('section', class_ = 'ticket-price')
                    for i in sub_parse.contents:
                        if type(i) == bs4.element.Tag and 'href' in i.attrs:
                            event_info.update([('Get Tickets', i.attrs['href'])])
                        elif type(i) == bs4.element.Tag:
                            for j in i.contents:
                                if type(j)== bs4.element.Tag and 'href' in j.attrs: 
                                    event_info.update([('Get Tickets', j.attrs['href'])])
                else:
                    event_info.update([('Get Tickets', event_info['Price/Admission'])])
                event_info.update([('Location Link', 'https://goo.gl/maps/d3G9381S4j1zD73s9')])
            
                self.info.append(event_info) 
            return self.info
        
        
        
        elif self.key == '{:04d}'.format(1): #texas ballet
            for link in self.pages:
                event_info = self.prd.copy()
                event_info.update([('Org Key',self.key)])
                event_info.update([('Event Key','{:06d}'.format(self.count))])
                self.count+=1
                s_page = requests.get(link)
                sub_soup = BeautifulSoup(s_page.text, 'html.parser')
                sub_par = sub_soup.find('div', class_='title')
                vTitle = (sub_par.h1.text.replace('\n','') if sub_par.h1 else contents[0]).strip()
                event_info.update([("Event Title",vTitle)])
                sub_par = sub_soup.find('div', class_='additional_info')
                vDates =(sub_par.p.text.replace('\n','') if sub_par.p else contents[0]).strip()
                event_info.update([("Date(s)",vDates)])
                sub_par = sub_soup.find("div", class_='main_title')
                desc=[]
                desc.append((sub_par.h3.text if sub_par.h3 else contetns[0]).strip())
                sub_par = sub_soup.find("div", class_='main_info')
                desc.append((sub_par.p.text if sub_par.p else contetns[0]).strip())
                sub_par = sub_soup.find_all("div", class_='single_info')
                for i in range(len(sub_par)):
                    desc.append((sub_par[i].h4.text.replace('\n','') if sub_par[i].h4 else contents[0]).strip())
                    desc.append(sub_par[i].p.text)
                sub_par = sub_soup.find_all("div", class_='additional_info')
                vdes = (sub_par[1].text.replace('\n','') if sub_par[1].text else contents[0]).strip()
                desc.append(vdes)
                des=''
                event_info.update([("Topline", desc.pop(0))])
                for i in desc:
                    if i == desc[len(desc)-1]:
                        event_info.update([("Venue",i)])
                    else: 
                        des+= i
                        des+="\n\n"
                event_info.update([("Event Description",des)])
                event_info.update([('Venue Info', "Performance Hall")])
                event_info.update([('Category', "Live Performance, Music, Dance, Ballet, Fine Arts, Sit Down")])
                sub_par = sub_soup.find_all("div", class_='single_person')
                ar=[]
                for i in range(len(sub_par)):
                    ar.append((sub_par[i].text.replace('\n',"") if sub_par[0].text else contents[0]).strip())
                art=""
                for i in ar:
                    art += i
                    art += "\n\n"
                event_info.update([("Staff/Artists",art)])
                sub_par = sub_soup.find(class_='image_holder_single')
                event_info.update([('Event Image URL', sub_par.contents[1].get('src'))])
                event_info.update([('URL',link)])
                
                #
                self.info.append(event_info)      
               
            spec = requests.get("https://texasballettheater.org/special-events/")
            spec.encoding = 'ISO-885901'
            sub_soup = BeautifulSoup(spec.text, 'lxml')
            page_list = sub_soup.find_all("div", class_="item same_height")
            bal_list = []
            for i in page_list:
                cont = i.contents[1]
                bal_list.append(cont['href'])
                
            for link in bal_list: #special events
                event_info = self.prd.copy()
                event_info.update([('Org Key',self.key)])
                event_info.update([('Event Key','{:06d}'.format(self.count))])
                self.count+=1
                s_page = requests.get(link)
                sub_soup = BeautifulSoup(s_page.text, 'html.parser')
                sub_par = sub_soup.find('div', class_='title')
                vTitle = (sub_par.h1.text.replace('\n','') if sub_par.h1 else contents[0]).strip()
                event_info.update([("Event Title",vTitle)])
                sub_par = sub_soup.find('section', class_='post_image')
                event_info.update([("Event Image URL",sub_par.contents[1].get("src"))])
                event_info.update([('Category', "Fine Arts,")])
                sub_par = sub_soup.find_all('div', class_='col-md-3 col-sm-6 col-12')
                dinfo = (sub_par[0].text.replace('  ',"").replace('\n',' ') if sub_par[0].text else contents[0]).strip()
                event_info.update([("Date(s)", dinfo)])
                sub_par.pop(0)
                tinfo = (sub_par[0].text.replace('  ',"").replace('\n',' ') if sub_par[0].text else contents[0]).strip()
                event_info.update([("Time(s)", tinfo)])
                sub_par.pop(0)
                vinfo = (sub_par[0].text.replace('  ',"").replace('\n',' ') if sub_par[0].text else contents[0]).strip()
                event_info.update([("Venue", vinfo)])
                event_info.update([('Venue Info', "Night Club")])
                sub_par.pop(0)
                pinfo = (sub_par[0].text.replace('  ',"").replace('\n',' ') if sub_par[0].text else contents[0]).strip()
                event_info.update([("Price/Admission", pinfo)])
                sub_par.pop(0)
                sub_par = sub_soup.find('section', class_='content_text')
                event_info.update([('Event Description', sub_par.text.replace("\n"," ").replace("\xa0",""))])
                event_info.update([('URL',link)])
                sub_par = sub_soup.find('a', class_='dark_btn')
                event_info.update([('Get Tickets',sub_par.get("href"))])
                
                
                self.info.append(event_info) 
            return self.info
        elif self.key == '{:04d}'.format(2):
            for i in self.pages:
                event_info = self.prd.copy()
                event_info.update([('Org Key',self.key)])
                event_info.update([('Event Key','{:06d}'.format(self.count))])
                self.count+=1
                event_info.update([('Event Image URL', i.contents[1].contents[1].get("data-src"))])
                
                
                
                if len(i.contents[3])<3 :
                    event_info.update([('Event Title',"NA")]) #title 
                    event_info.update([('Event Description',"Event data TBD")])
                elif type(i.contents[3].contents[1].h5) == bs4.element.Tag:
                    event_info.update([('Event Title',i.contents[3].contents[1].h5.text)]) #title
                    l =i.contents[3].find_all('p')
                    event_info.update([('Topline',l[0].text)]) #topline
                    event_info.update([('Event Description',l[1].text)]) # desc
                    event_info.update([('Time(s)',l[2].text)]) #time
                elif type(i.contents[3].p.text.split("\n",1)[0]) == str:
                    event_info.update([("Event Title",i.contents[3].p.text.split("\n",1)[0])])
                    event_info.update([("Event Description",i.contents[3].p.text.split("\n",1)[1])])
                    event_info.update([("Time(s)",i.contents[3].find_all('p')[1].text)])
               
                event_info.update([('Category', "Virtual, Performance, Music, Dance, Ballet, Fine Arts, Sit Down")])

                event_info.update([('URL', bonus)])
                
             #   event_info.update([()])
              #  event_info.update([()])
               # event_info.update([()])
                #event_info.update([()])
                self.info.append(event_info)
            
            return self.info
        elif self.key == '{:04d}'.format(3):
            for link in self.pages:
                event_info = self.prd.copy()
                event_info.update([('Org Key',self.key)])
                event_info.update([('Event Key','{:06d}'.format(self.count))])
                self.count+=1
                s_page = requests.get(link)
                sub_soup = BeautifulSoup(s_page.text, 'html.parser')
                event_info.update([("Event Title",sub_soup.find('h1', class_='title').text)])
                event_info.update([("Event Image URL",("http:"+str(sub_soup.find("span",class_="header_thumbnail").contents[1]).split("\"",2)[1]))])
                event_info.update([("URL",link)])
                event_info.update([("Venue",sub_soup.find('span', class_='meta').text)])
                event_info.update([("Event Description",sub_soup.find("div",class_="entry-content").p.contents[0].get("content").replace("\r","").replace("\xa0","").replace("\xa0A","").replace("\n"," ")
)])
                
                
                self.info.append(event_info)
            return self.info
        else:
            print("Invalid Key")
            return[]    

In [4]:
class af_org:
    
    id_count=0
    orgs ={}
    org_data =pd.DataFrame()
    
    def __init__(self, name):
        self.name = name
        self.sUrl =""
        self.ssurl = ""
        self.ID = '{:04d}'.format(af_org.id_count)
        af_org.id_count+=1
        self.sub_pg = []
        self.data = []
        self.info= {"Org ID": self.ID,
                    "Org Name": self.name,
                    "Org URL": self.sUrl}
        af_org.orgs[self.ID] = self.name
        
    def set_sUrl(self, sUrl):
        try: 
            requests.get(sUrl) 
        except InvalidSchema as exception:
            print("URL is not complete: Please try again")
        except requests.ConnectionError as exception:
            print("URL does not exist on Internet. Please try Annother URL") 
        except MissingSchema as exception:
            print("URL is not complete: Please try again") 
        self.sUrl = sUrl
        self.info['Org URL'] = sUrl
        
    def update(self, data):
        self.data = data
        for i in range(len(af_org.org_data)):
            if af_org.org_data.at[i,"Org Key"]=='{:04d}'.format(int(self.ID)):
                af_org.org_data = af_org.org_data.drop(i)
        af_org.org_data = af_org.org_data.append(df.from_records(self.data), ignore_index = True)
        af_org.org_data = af_org.org_data.sort_values(by=['Org Key'])

In [5]:
dada = af_org("Dada Dallas")
dada.set_sUrl("https://www.dadadallas.com/calendar/")
dada.sub_pg = crawly(dada.ID, dada.sUrl).crawl()
dada.update(creepy(dada.ID, dada.sub_pg).creep())

In [6]:
tbt = af_org("Texas Ballet Theater")

In [7]:
tbt.set_sUrl('https://texasballettheater.org/20-21season/')
tbt.ssUrl = "https://texasballettheater.org/special-events/"
tbt.sub_pg = crawly(tbt.ID, tbt.sUrl).crawl()
tbt.update(creepy(tbt.ID, tbt.sub_pg).creep())


In [8]:
the = af_org("Theater3")
the.set_sUrl("https://www.theatre3dallas.com/shows-tickets/")
the.ID = '{:04d}'.format(2)
the.sub_pg = crawly(the.ID, the.sUrl).crawl()
the.update(creepy(the.ID, the.sub_pg).creep(the.sUrl))

In [9]:
LCC = af_org("Latino Cultural Center")
LCC.ID = '{:04d}'.format(3)
LCC.set_sUrl("http://lcc.dallasculture.org/programs/event-calendar/")
LCC.sub_pg = crawly(LCC.ID, LCC.sUrl).crawl()
LCC.update(creepy(LCC.ID, LCC.sub_pg).creep())

In [10]:
LCC.update(creepy(LCC.ID, LCC.sub_pg).creep())


In [11]:
tbt.data

[{'Org Key': '0001',
  'Event Key': '000000',
  'Event Title': 'The Nutcracker',
  'Topline': 'At Midnight, the Clock Strikes Magic',
  'Headliner': '',
  'Openers': '',
  'Date(s)': 'Dec 14 - Dec 26, 2020',
  'Time(s)': '',
  'Price/Admission': '',
  'Age Restriction': '',
  'Event Description': 'Experience the most magical tradition of the holiday season from the comfort of your own home. Ben Stevenson’s The Nutcracker is available online for the first time ever, and when you purchase a ticket, you’ll also receive access to The Nutty Nutcracker. From snowflakes to sweets, The Nutcracker will dazzle and delight to become your favorite joy of the season. This video was filmed in 2012, so you may see a few old favorites onstage!\n\n',
  'Staff/Artists': 'Choreographer                                                                            Ben Stevenson, O.B.E.\n\nComposer                                                                            Pyotr Ilyich Tchaikovsky\n\nRun Time  

In [12]:
af_org.org_data

,Org Key,Event Key,Event Title,Topline,Headliner,Openers,Date(s),Time(s),Price/Admission,Age Restriction,...,Venue Info,Street Name,Address Line 2,City,State,Postal Code,Event Image URL,Location Link,Get Tickets,URL
0,0000,000000,"Vundabar - Tickets - Club Dada - Dallas, TX - ...",Margin Walker presents,Vundabar,NA,"Wed, December 2, 2020",7 PM,$15.00 - $18.00,All Ages,...,Night Club,2720 Elm St,",",Dallas,TX,75226,https://img-dev.evbuc.com/https%3A%2F%2Fcdn.ev...,https://goo.gl/maps/d3G9381S4j1zD73s9,http://eventbrite.com/e/vundabar-new-date-club...,https://www.dadadallas.com/e/vundabar-10465726...
1,0000,000001,"Azizi Gibson - Tickets - Club Dada - Dallas, T...",Margin Walker presents,Azizi Gibson,NA,"Fri, December 4, 2020",7 PM,Sales Ended,All Ages,...,Night Club,2720 Elm St,",",Dallas,TX,75226,https://img-dev.evbuc.com/https%3A%2F%2Fcdn.ev...,https://goo.gl/maps/d3G9381S4j1zD73s9,Sales Ended,https://www.dadadallas.com/e/azizi-gibson-9319...
2,0000,000002,SURFER BLOOD • Winter - Tickets - Club Dada - ...,presented by pofTX,Surfer Blood,Winter,"Wed, February 3, 2021",7 PM,$14.99,All Ages,...,Night Club,2720 Elm St,",",Dallas,TX,75226,https://img-dev.evbuc.com/https%3A%2F%2Fcdn.ev...,https://goo.gl/maps/d3G9381S4j1zD73s9,https://www.eventbrite.com/e/surfer-blood-wint...,https://www.dadadallas.com/e/surfer-blood-wint...
3,0000,000003,NateWantsToBattle • Vespera - Tickets - Club D...,Margin Walker presents,NateWantsToBattle,Vespera,"Mon, March 29, 2021",7 PM,Sales Ended,All Ages,...,Night Club,2720 Elm St,",",Dallas,TX,75226,https://img-dev.evbuc.com/https%3A%2F%2Fcdn.ev...,https://goo.gl/maps/d3G9381S4j1zD73s9,Sales Ended,https://www.dadadallas.com/e/natewantstobattle...
4,0000,000004,"Anti-Flag - Tickets - Club Dada - Dallas, TX -...",Margin Walker presents,Anti-Flag,NA,"Tue, May 18, 2021",7 PM,$18.00 - $20.00,All Ages,...,Night Club,2720 Elm St,",",Dallas,TX,75226,https://img-dev.evbuc.com/https%3A%2F%2Fcdn.ev...,https://goo.gl/maps/d3G9381S4j1zD73s9,http://eventbrite.com/e/anti-flag-new-date-clu...,https://www.dadadallas.com/e/anti-flag-1050000...
5,0000,000005,Armor For Sleep - Tickets - Club Dada - Dallas...,Margin Walker presents,Armor For Sleep,NA,"Thu, July 15, 2021",7 PM,$25.00 - $28.00,All Ages,...,Night Club,2720 Elm St,",",Dallas,TX,75226,https://img-dev.evbuc.com/https%3A%2F%2Fcdn.ev...,https://goo.gl/maps/d3G9381S4j1zD73s9,http://eventbrite.com/e/armor-for-sleep-club-d...,https://www.dadadallas.com/e/armor-for-sleep-9...
11,0001,000003,Bartok | Image | Imbue,Three Iconic Pieces. One Incredible Night.,,,"Mar 05 - Mar 07, 2021",,,,...,Performance Hall,,",",,,,https://texasballettheater.org/wp-content/uplo...,,,https://texasballettheater.org/perfomance/bart...
10,0001,000005,A Masked Ballet,,,,"Date December 5, 2020",Time 7:00 pm,Price $100 per ticket; sponsor opportunities...,,...,Night Club,,",",,,,https://texasballettheater.org/wp-content/uplo...,,https://texasballettheater.networkforgood.com/...,https://texasballettheater.org/events/a-masked...
9,0001,000004,Beauty & The Beast,True Love Blooms from Within,,,"May 14 - May 23, 2021",,,,...,Performance Hall,,",",,,,https://texasballettheater.org/wp-content/uplo...,,,https://texasballettheater.org/perfomance/beau...
7,0001,000000,The Nutcracker,"At Midnight, the Clock Strikes Magic",,,"Dec 14 - Dec 26, 2020",,,,...,Performance Hall,,",",,,,https://texasballettheater.org/wp-content/uplo...,,,https://texasballettheater.org/perfomance/the-...


In [13]:
df.to_csv(af_org.org_data, "M3.csv", sep = ",", index = False)

In [162]:
DAD = af_org("Latino Cultural Center")
DAD.ID = '{:04d}'.format(4)
DAD.set_sUrl("https://artandseek.org/calendar/organization/897/dallas-museum-of-art")
DAD.sub_pg = crawly(DAD.ID, DAD.sUrl).crawl()
#DAD.update(creepy(DAD.ID, DAD.sub_pg).creep())

In [163]:
DAD.sub_pg

['https://www.dallasartsdistrict.org/events/not-visible-to-the-naked-eye-inside-a-senufo-helmet-mask/',
 'https://www.dallasartsdistrict.org/events/beili-liu-lure-and-each-and-every-exhibitions/',
 'https://www.dallasartsdistrict.org/events/flores-mexicanas-women-in-modern-mexican-art/',
 'https://www.dallasartsdistrict.org/events/frans-hals-detecting-a-decade/',
 'https://www.dallasartsdistrict.org/events/resistrelease/',
 'https://www.dallasartsdistrict.org/events/barry-x-ball-remaking-sculpture/',
 'https://www.dallasartsdistrict.org/events/mygration/',
 'https://www.dallasartsdistrict.org/events/2020-nasher-prize-laureate-michael-rakowitz/',
 'https://www.dallasartsdistrict.org/events/for-a-dreamer-of-houses/',
 'https://www.dallasartsdistrict.org/events/klyde-warren-parks-movies-in-the-park/']

In [31]:
req = requests.get("https://api.akk.io/api?flow_key=X3WEIyQKtJMJc5oScFJF%2F1&api_key=79c21d4a-fbb2-4b64-8739-52f5b6a6cd04&sample=true&data=%5B%7B%22Org%20Key%22%3A%220%22%2C%22Event%20Key%22%3A%220%22%2C%22Event%20Title%22%3A%22Vundabar%20-%20Tickets%20-%20Club%20Dada%20-%20Dallas%2C%20TX%20-%20December%202nd%2C%202020%22%2C%22Topline%22%3A%22Margin%20Walker%20presents%22%2C%22Headliner%22%3A%22Vundabar%22%2C%22Openers%22%3A%22%22%2C%22Date(s)%22%3A%22Wed%2C%20December%202%2C%202020%22%2C%22Time(s)%22%3A%227%3A00%20PM%22%2C%22Price%2FAdmission%22%3A%22%2415.00%20-%20%2418.00%22%2C%22Age%20Restriction%22%3A%22All%20Ages%22%2C%22Event%20Description%22%3A%22Margin%20Walker%20presents%20Vundabar%20at%20Club%20Dada%20%20%22%2C%22Staff%2FArtists%22%3A%22%22%2C%22Venue%22%3A%22Club%20Dada%22%2C%22Venue%20Info%22%3A%22Night%20Club%22%2C%22Street%20Name%22%3A%222720%20Elm%20St%22%2C%22Address%20Line%202%22%3A%22%2C%22%2C%22City%22%3A%22Dallas%22%2C%22State%22%3A%22TX%22%2C%22Postal%20Code%22%3A%2275226%22%2C%22Event%20Image%20URL%22%3A%22https%3A%2F%2Fimg-dev.evbuc.com%2Fhttps%253A%252F%252Fcdn.evbuc.com%252Fimages%252F101013404%252F231623301463%252F1%252Foriginal.20200514-210026%3Fauto%3Dcompress%26fit%3Dclip%26h%3D%26w%3D300%26s%3D98fd396465362c2d7218352ebdeb70ef%22%2C%22Location%20Link%22%3A%22https%3A%2F%2Fgoo.gl%2Fmaps%2Fd3G9381S4j1zD73s9%22%2C%22Get%20Tickets%22%3A%22http%3A%2F%2Feventbrite.com%2Fe%2Fvundabar-new-date-club-dada-tickets-76275950421%22%2C%22URL%22%3A%22https%3A%2F%2Fwww.dadadallas.com%2Fe%2Fvundabar-104657262690%2F%22%7D%2C%7B%22Org%20Key%22%3A%220%22%2C%22Event%20Key%22%3A%221%22%2C%22Event%20Title%22%3A%22Azizi%20Gibson%20-%20Tickets%20-%20Club%20Dada%20-%20Dallas%2C%20TX%20-%20December%204th%2C%202020%22%2C%22Topline%22%3A%22Margin%20Walker%20presents%22%2C%22Headliner%22%3A%22Azizi%20Gibson%22%2C%22Openers%22%3A%22%22%2C%22Date(s)%22%3A%22Fri%2C%20December%204%2C%202020%22%2C%22Time(s)%22%3A%227%3A00%20PM%22%2C%22Price%2FAdmission%22%3A%22Sales%20Ended%22%2C%22Age%20Restriction%22%3A%22All%20Ages%22%2C%22Event%20Description%22%3A%22Margin%20Walker%20presents%20Azizi%20Gibson%20at%20Club%20Dada%20%22%2C%22Staff%2FArtists%22%3A%22%22%2C%22Venue%22%3A%22Club%20Dada%22%2C%22Venue%20Info%22%3A%22Night%20Club%22%2C%22Street%20Name%22%3A%222720%20Elm%20St%22%2C%22Address%20Line%202%22%3A%22%2C%22%2C%22City%22%3A%22Dallas%22%2C%22State%22%3A%22TX%22%2C%22Postal%20Code%22%3A%2275226%22%2C%22Event%20Image%20URL%22%3A%22https%3A%2F%2Fimg-dev.evbuc.com%2Fhttps%253A%252F%252Fcdn.evbuc.com%252Fimages%252F97976078%252F231623301463%252F1%252Foriginal.20200403-204954%3Fauto%3Dcompress%26fit%3Dclip%26h%3D%26w%3D300%26s%3D763d72aed82c76e5480319e063ea917d%22%2C%22Location%20Link%22%3A%22https%3A%2F%2Fgoo.gl%2Fmaps%2Fd3G9381S4j1zD73s9%22%2C%22Get%20Tickets%22%3A%22Sales%20Ended%22%2C%22URL%22%3A%22https%3A%2F%2Fwww.dadadallas.com%2Fe%2Fazizi-gibson-93194333761%2F%22%7D%2C%7B%22Org%20Key%22%3A%220%22%2C%22Event%20Key%22%3A%222%22%2C%22Event%20Title%22%3A%22SURFER%20BLOOD%20%E2%80%A2%20Winter%20-%20Tickets%20-%20Club%20Dada%20-%20Dallas%2C%20TX%20-%20February%203rd%2C%202021%22%2C%22Topline%22%3A%22presented%20by%20pofTX%22%2C%22Headliner%22%3A%22Surfer%20Blood%22%2C%22Openers%22%3A%22Winter%22%2C%22Date(s)%22%3A%22Wed%2C%20February%203%2C%202021%22%2C%22Time(s)%22%3A%227%3A00%20PM%22%2C%22Price%2FAdmission%22%3A%22%2414.99%20%22%2C%22Age%20Restriction%22%3A%22All%20Ages%22%2C%22Event%20Description%22%3A%22SURFER%20BLOOD%27s%20rescheduled%20show%20from%20Not%20So%20Fun%20Wknd%202020.%20Bringing%20WINTER%20with%20them%20at%20it%20is%20in%20winter.%20It%20will%20probably%20be%20so%20cold%20out.%20Surfer%20Blood%20have%20just%20finished%20recording%20their%205th%20album%2C%20which%20will%20be%20coming%20out%20on%20July%2017%2C%202020%2C%20alongside%20a%2010th%20anniversary%20reissue%20of%20their%20debut%20album%20Astro%20Coast.%20Surfer%20Blood%27s%20music%20is%20renowned%20for%20catchy%20hooks%20and%20sonic%20detail.%20One%20of%20the%20best%20indie%20guitar%20bands%20around%2C%20founding%20members%20John%20Paul%20Pitts%20and%20Tyler%20Schwarz%20are%20joined%20by%20guitarist%20Michael%20McCleary%20and%20bassist%20Lindsey%20Mills%2C%20all%20four%20alumni%20of%20the%20same%20high%20school%20in%20West%20Palm%20Beach%2C%20Florida.Surfer%20Blood%20are%20the%20cleanest%20and%20nicest%20band%20in%20existence.%22%2C%22Staff%2FArtists%22%3A%22%22%2C%22Venue%22%3A%22Club%20Dada%22%2C%22Venue%20Info%22%3A%22Night%20Club%22%2C%22Street%20Name%22%3A%222720%20Elm%20St%22%2C%22Address%20Line%202%22%3A%22%2C%22%2C%22City%22%3A%22Dallas%22%2C%22State%22%3A%22TX%22%2C%22Postal%20Code%22%3A%2275226%22%2C%22Event%20Image%20URL%22%3A%22https%3A%2F%2Fimg-dev.evbuc.com%2Fhttps%253A%252F%252Fcdn.evbuc.com%252Fimages%252F98116308%252F277453266158%252F1%252Foriginal.20200406-175206%3Fauto%3Dcompress%26fit%3Dclip%26h%3D%26w%3D300%26s%3Da158ea94eb421b69fdc316f017ef54b4%22%2C%22Location%20Link%22%3A%22https%3A%2F%2Fgoo.gl%2Fmaps%2Fd3G9381S4j1zD73s9%22%2C%22Get%20Tickets%22%3A%22https%3A%2F%2Fwww.eventbrite.com%2Fe%2Fsurfer-blood-winter-tickets-102104597600%22%2C%22URL%22%3A%22https%3A%2F%2Fwww.dadadallas.com%2Fe%2Fsurfer-blood-winter-102104597600%2F%22%7D%5D")

In [32]:
req.json()

[{'Category': 'Live Music, Night Life, Performance',
  'Probability Category is Virtual, Performance, Music, Dance, Ballet, Fine Arts, Sit Down': 0.0009470058721490204,
  'Probability Category is Live Music, Night Life, Performance': 0.9961950778961182,
  'Probability Category is Live Performance, Music, Dance, Ballet, Fine Arts, Sit Down': 0.0007114848704077303,
  'Probability Category is Fine Arts,': 0.0008495263173244894,
  'Probability Category is ': 0.001296916278079152},
 {'Category': 'Live Music, Night Life, Performance',
  'Probability Category is Virtual, Performance, Music, Dance, Ballet, Fine Arts, Sit Down': 0.0018935039406642318,
  'Probability Category is Live Music, Night Life, Performance': 0.9961223006248474,
  'Probability Category is Live Performance, Music, Dance, Ballet, Fine Arts, Sit Down': 0.0006586697418242693,
  'Probability Category is Fine Arts,': 0.0005119654815644026,
  'Probability Category is ': 0.0008135456591844559},
 {'Category': 'Live Music, Night Li

In [25]:
parameters = {
    "flow_key" : 'X3WEIyQKtJMJc5oScFJF/1',
    "api_key" : '79c21d4a-fbb2-4b64-8739-52f5b6a6cd04',
    "data" : af_org.org_data}

In [28]:
def jprint(obj):
    text = json.dumps(obj, sort_keys = True, indent=4)
    print(text)


In [ ]:
curl -G https://api.akk.io/api \
-d flow_key='X3WEIyQKtJMJc5oScFJF/1' \
-d api_key='79c21d4a-fbb2-4b64-8739-52f5b6a6cd04' \
-d data='[{"Org Key":"0","Event Key":"0","Event Title":"Vundabar - Tickets - Club Dada - Dallas, TX - December 2nd, 2020","Topline":"Margin Walker presents","Headliner":"Vundabar","Openers":"","Date(s)":"Wed, December 2, 2020","Time(s)":"7:00 PM","Price/Admission":"$15.00 - $18.00","Age Restriction":"All Ages","Event Description":"Margin Walker presents Vundabar at Club Dada  ","Staff/Artists":"","Venue":"Club Dada","Venue Info":"Night Club","Street Name":"2720 Elm St","Address Line 2":",","City":"Dallas","State":"TX","Postal Code":"75226","Event Image URL":"https://img-dev.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F101013404%2F231623301463%2F1%2Foriginal.20200514-210026?auto=compress&fit=clip&h=&w=300&s=98fd396465362c2d7218352ebdeb70ef","Location Link":"https://goo.gl/maps/d3G9381S4j1zD73s9","Get Tickets":"http://eventbrite.com/e/vundabar-new-date-club-dada-tickets-76275950421","URL":"https://www.dadadallas.com/e/vundabar-104657262690/"},{"Org Key":"0","Event Key":"1","Event Title":"Azizi Gibson - Tickets - Club Dada - Dallas, TX - December 4th, 2020","Topline":"Margin Walker presents","Headliner":"Azizi Gibson","Openers":"","Date(s)":"Fri, December 4, 2020","Time(s)":"7:00 PM","Price/Admission":"Sales Ended","Age Restriction":"All Ages","Event Description":"Margin Walker presents Azizi Gibson at Club Dada ","Staff/Artists":"","Venue":"Club Dada","Venue Info":"Night Club","Street Name":"2720 Elm St","Address Line 2":",","City":"Dallas","State":"TX","Postal Code":"75226","Event Image URL":"https://img-dev.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F97976078%2F231623301463%2F1%2Foriginal.20200403-204954?auto=compress&fit=clip&h=&w=300&s=763d72aed82c76e5480319e063ea917d","Location Link":"https://goo.gl/maps/d3G9381S4j1zD73s9","Get Tickets":"Sales Ended","URL":"https://www.dadadallas.com/e/azizi-gibson-93194333761/"},{"Org Key":"0","Event Key":"2","Event Title":"SURFER BLOOD • Winter - Tickets - Club Dada - Dallas, TX - February 3rd, 2021","Topline":"presented by pofTX","Headliner":"Surfer Blood","Openers":"Winter","Date(s)":"Wed, February 3, 2021","Time(s)":"7:00 PM","Price/Admission":"$14.99 ","Age Restriction":"All Ages","Event Description":"SURFER BLOOD's rescheduled show from Not So Fun Wknd 2020. Bringing WINTER with them at it is in winter. It will probably be so cold out. Surfer Blood have just finished recording their 5th album, which will be coming out on July 17, 2020, alongside a 10th anniversary reissue of their debut album Astro Coast. Surfer Blood's music is renowned for catchy hooks and sonic detail. One of the best indie guitar bands around, founding members John Paul Pitts and Tyler Schwarz are joined by guitarist Michael McCleary and bassist Lindsey Mills, all four alumni of the same high school in West Palm Beach, Florida.Surfer Blood are the cleanest and nicest band in existence.","Staff/Artists":"","Venue":"Club Dada","Venue Info":"Night Club","Street Name":"2720 Elm St","Address Line 2":",","City":"Dallas","State":"TX","Postal Code":"75226","Event Image URL":"https://img-dev.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F98116308%2F277453266158%2F1%2Foriginal.20200406-175206?auto=compress&fit=clip&h=&w=300&s=a158ea94eb421b69fdc316f017ef54b4","Location Link":"https://goo.gl/maps/d3G9381S4j1zD73s9","Get Tickets":"https://www.eventbrite.com/e/surfer-blood-winter-tickets-102104597600","URL":"https://www.dadadallas.com/e/surfer-blood-winter-102104597600/"}]'